# Data preprocessing

# 1. Load data

In [160]:
INPUT_FOLDER = './channels/'
OUTPUT_FOLDER = './data/preprocessed_v1/'
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from datetime import datetime, timezone

In [101]:
# make a list of all the channel files
channels = []
for filename in tqdm(os.listdir(INPUT_FOLDER), desc='channels', unit='filenames'):
    if filename.endswith('.csv'):
        channels.append(filename)
print(len(channels))

channels: 100%|██████████| 300/300 [00:00<?, ?filenames/s]

299


In [102]:
#   method to read a channel file to a dataframe
def read_channel(filename):
    dtype_dict = {'id': 'int64', 'views': 'float64', 'message': 'str'}
    df = pd.read_csv(os.path.join(INPUT_FOLDER, filename), index_col='Unnamed: 0', dtype=dtype_dict, parse_dates=['date'])
    df['handle'] = filename[:-4]
    return df

In [103]:
df = read_channel(channels[0])
df.head()

,id,date,views,reactions,to_id,fwd_from,message,type,duration,handle
0,12602,2022-12-19 13:05:23+00:00,3645.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1261603870),NaN,🇵🇰🇦🇫 Исламабад сделал ставку на афганских тали...,photo,NaN,Abbasdjuma
1,12601,2022-12-19 09:52:21+00:00,5831.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1261603870),NaN,"🕯️ 6 лет назад, 19 декабря 2016 года, в резуль...",photo,NaN,Abbasdjuma
2,12600,2022-12-19 09:18:53+00:00,3944.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1261603870),NaN,🇮🇷🇷🇺 Глава МИД Ирана Хосейн Амир Абдоллахиян с...,photo,NaN,Abbasdjuma
3,12599,2022-12-19 08:32:39+00:00,2970.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1261603870),"MessageFwdHeader(date=datetime.datetime(2022, ...",Наши Друзья открыли \nсбор для одного из Доне...,photo,NaN,Abbasdjuma
4,12598,2022-12-18 21:41:25+00:00,4993.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1261603870),NaN,"🇷🇺 Сегодня, 19 декабря в России празднуют День...",photo,NaN,Abbasdjuma


In [130]:
print(type(df['date'][0]))
print(df['date'][0]-df['date'][1])

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 01:51:01


In [106]:
df['reactions'][0]

"MessageReactions(results=[ReactionCount(reaction='🤔', count=24, chosen=False), ReactionCount(reaction='👍', count=7, chosen=False), ReactionCount(reaction='🔥', count=1, chosen=False), ReactionCount(reaction='😁', count=1, chosen=False), ReactionCount(reaction='🤮', count=1, chosen=False)], min=False, can_see_list=False, recent_reactions=[])"

In [107]:
# Define a function to extract reaction data from the string
def extract_reactions(row):
    pattern = r"ReactionCount\(reaction='(\S+)', count=(\d+),"
    try:
        matches = re.findall(pattern, row)
    except TypeError:
        print(row)
    # matches = re.findall(pattern, row)
    return {reaction: int(count) for reaction, count in matches}

In [108]:
row_reacts = extract_reactions(df['reactions'][0])
row_reacts['🤔']

24

In [111]:
def save_channel(df, filename):
  df.to_csv(os.path.join(OUTPUT_FOLDER, filename))

In [120]:
# Define a function to extract reaction data from the string and
def extract_reactions(row):
    if pd.isna(row):
        return pd.Series(dtype='float64')
    pattern = r"ReactionCount\(reaction='(\S+)', count=(\d+),"
    try:
        matches = re.findall(pattern, row)
    except TypeError:
        print(row)
        raise Exception(f"Reactions regex error:{e}");
    if not matches:
        # No matches found, return an empty Series
        return pd.Series(dtype='float64')
    reactions_data = {react: count for react, count in matches}
    try:
        return pd.Series(reactions_data, dtype='float64')
    except e:
        print(row)
        print(matches)
        print(reactions_data)
        raise Exception(f"Reactions processing error:{e}");

In [128]:
def process_reactions_df(df):
    # Apply the function to create new columns
    df = pd.concat([df, df['reactions'].progress_apply(extract_reactions)], axis=1)
    df.drop(columns=['reactions'], inplace=True)
    return df

In [123]:
df.head()

,id,date,views,to_id,fwd_from,message,type,duration,handle,🤬,😱,👍,😁,❤,🔥,🤯,🤔,🥰,👏
0,5968,2022-12-18 17:44:04+00:00,69423.0,PeerChannel(channel_id=1099760830),NaN,Донецк в огне. Больницу им. Калинина эвакуирую...,text,NaN,akashevarova,3827.0,480.0,139.0,106.0,47.0,24.0,18.0,17.0,3.0,NaN
1,5967,2022-12-18 15:53:03+00:00,92385.0,PeerChannel(channel_id=1099760830),NaN,"А вот еще одна проблема, про которую узнала от...",text,NaN,akashevarova,157.0,176.0,6593.0,437.0,838.0,201.0,50.0,31.0,25.0,76.0
2,5966,2022-12-18 10:11:34+00:00,90301.0,PeerChannel(channel_id=1099760830),NaN,Вчера в Луганске посетили госпиталь. 1195 паци...,text,NaN,akashevarova,19.0,1.0,3691.0,71.0,1286.0,74.0,NaN,8.0,7.0,129.0
3,5965,2022-12-17 22:45:28+00:00,173857.0,PeerChannel(channel_id=1099760830),NaN,"Ой, ну ладно вам. Мы живы. ПВО сработало\n\nht...",text,NaN,akashevarova,22.0,53.0,1063.0,139.0,272.0,NaN,7.0,40.0,NaN,NaN
4,5964,2022-12-17 12:40:48+00:00,474103.0,PeerChannel(channel_id=1099760830),NaN,Поговорили с Андреем Турчаком @turchak_andrey ...,text,NaN,akashevarova,27.0,4.0,1626.0,50.0,9292.0,111.0,5.0,22.0,135.0,385.0


In [124]:
type(df['🤔'][0])

numpy.float64

In [132]:
df['to_id'][0]

'PeerChannel(channel_id=1099760830)'

In [133]:
# function to extract to_id from the string
def extract_to_id(row):
    # Define a pattern to extract the channel ID
    pattern = r'channel_id=(\d+)'

    # Use re.search to find the match in the string
    match = re.search(pattern, row)

    # Check if a match is found and extract the channel ID
    if match:
        return int(match.group(1))
    else:
        return np.NaN


In [134]:
extract_to_id(df['to_id'][0])

1099760830

In [135]:
def extract_to_id_df(df):
    # Apply the function to create the new column
    df['to_id'] = df['to_id'].progress_apply(extract_to_id)
    return df

In [138]:
df.value_counts('to_id')

to_id
PeerChannel(channel_id=1099760830)    5880
Name: count, dtype: int64

In [140]:
extract_to_id_df(df)

2/299 akashevarova: 100%|██████████| 5880/5880 [00:00<00:00, 237041.49rows/s]


,id,date,views,to_id,fwd_from,message,type,duration,handle,🤬,😱,👍,😁,❤,🔥,🤯,🤔,🥰,👏
0,5968,2022-12-18 17:44:04+00:00,69423.0,1099760830,NaN,Донецк в огне. Больницу им. Калинина эвакуирую...,text,NaN,akashevarova,3827.0,480.0,139.0,106.0,47.0,24.0,18.0,17.0,3.0,NaN
1,5967,2022-12-18 15:53:03+00:00,92385.0,1099760830,NaN,"А вот еще одна проблема, про которую узнала от...",text,NaN,akashevarova,157.0,176.0,6593.0,437.0,838.0,201.0,50.0,31.0,25.0,76.0
2,5966,2022-12-18 10:11:34+00:00,90301.0,1099760830,NaN,Вчера в Луганске посетили госпиталь. 1195 паци...,text,NaN,akashevarova,19.0,1.0,3691.0,71.0,1286.0,74.0,NaN,8.0,7.0,129.0
3,5965,2022-12-17 22:45:28+00:00,173857.0,1099760830,NaN,"Ой, ну ладно вам. Мы живы. ПВО сработало\n\nht...",text,NaN,akashevarova,22.0,53.0,1063.0,139.0,272.0,NaN,7.0,40.0,NaN,NaN
4,5964,2022-12-17 12:40:48+00:00,474103.0,1099760830,NaN,Поговорили с Андреем Турчаком @turchak_andrey ...,text,NaN,akashevarova,27.0,4.0,1626.0,50.0,9292.0,111.0,5.0,22.0,135.0,385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875,6,2017-01-30 19:58:34+00:00,765.0,1099760830,NaN,Только что выходила в Инстаграмм в прямой эфир...,text,NaN,akashevarova,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5876,5,2017-01-30 18:46:14+00:00,827.0,1099760830,NaN,"Да, поправка - МЕЧ. Я грамотная, понятно!",text,NaN,akashevarova,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5877,4,2017-01-30 18:30:54+00:00,1081.0,1099760830,NaN,"Была сегодня на ""Царьграде"" по теме хакеров из...",text,NaN,akashevarova,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5878,3,2017-01-30 18:08:23+00:00,1487.0,1099760830,NaN,"Вот решила тоже создать телеграмм-канал. То, ч...",text,NaN,akashevarova,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df['fwd_from'][20]

'MessageFwdHeader(date=datetime.datetime(2022, 12, 12, 7, 47, 22, tzinfo=datetime.timezone.utc), imported=False, from_id=PeerChannel(channel_id=1301168156), from_name=None, channel_post=1506, post_author=None, saved_from_peer=None, saved_from_msg_id=None, psa_type=None)'

In [147]:
def check_columns(df_list, columns_to_check):
    result = {}

    for idx, df in enumerate(df_list):
        present_columns = [col for col in columns_to_check if col in df.columns]
        result[idx] = present_columns

    return result

# Specify the columns you want to check
columns_to_check = ['channel_name', 'frw_from_title', 'frw_from_name', 'msg_entity']

In [148]:
# Call the function with your list of DataFrames and columns to check
present_columns_result = check_columns((read_channel(fn) for fn in channels), columns_to_check)

C:\Users\IvarY\AppData\Local\Temp\ipykernel_11356\3396395887.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(INPUT_FOLDER, filename), index_col='Unnamed: 0', dtype=dtype_dict, parse_dates=['date'])
C:\Users\IvarY\AppData\Local\Temp\ipykernel_11356\3396395887.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(INPUT_FOLDER, filename), index_col='Unnamed: 0', dtype=dtype_dict, parse_dates=['date'])
C:\Users\IvarY\AppData\Local\Temp\ipykernel_11356\3396395887.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(INPUT_FOLDER, filename), index_col='Unnamed: 0', dtype=dtype_dict, parse_dates=['date'])
C:\Users\IvarY\AppData\Local\Temp\ipykernel_11356\3396395887.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on i

In [149]:
# Print the result
for df_name, present_columns in present_columns_result.items():
    if present_columns:
        print(f'{df_name}: {present_columns}')

4: ['frw_from_title', 'frw_from_name', 'msg_entity']
28: ['frw_from_title', 'frw_from_name', 'msg_entity']
42: ['frw_from_title', 'frw_from_name', 'msg_entity']
54: ['frw_from_title', 'frw_from_name', 'msg_entity']
64: ['frw_from_title', 'frw_from_name', 'msg_entity']
72: ['frw_from_title', 'frw_from_name', 'msg_entity']
76: ['frw_from_title', 'frw_from_name', 'msg_entity']
112: ['frw_from_title', 'frw_from_name', 'msg_entity']
128: ['frw_from_title', 'frw_from_name', 'msg_entity']
140: ['frw_from_title', 'frw_from_name', 'msg_entity']
146: ['frw_from_title', 'frw_from_name', 'msg_entity']
155: ['frw_from_title', 'frw_from_name', 'msg_entity']
165: ['frw_from_title', 'frw_from_name', 'msg_entity']
169: ['frw_from_title', 'frw_from_name', 'msg_entity']
198: ['frw_from_title', 'frw_from_name', 'msg_entity']
246: ['frw_from_title', 'frw_from_name', 'msg_entity']
253: ['frw_from_title', 'frw_from_name', 'msg_entity']
260: ['frw_from_title', 'frw_from_name', 'msg_entity']
266: ['frw_from_ti

In [243]:
test_df = read_channel(channels[4])
test_df

,id,date,views,reactions,to_id,fwd_from,message,type,duration,frw_from_title,frw_from_name,msg_entity,handle
0,15377,2022-12-04 10:26:53+00:00,173.0,NaN,PeerChannel(channel_id=1109279145),"MessageFwdHeader(date=datetime.datetime(2022, ...",🚇БКЛ полноценно заработает уже в первом кварта...,photo,NaN,Дептранс Москвы,DtRoad,[<telethon.tl.types.MessageEntityItalic object...,akitilop
1,15376,2022-12-03 19:32:06+00:00,528.0,NaN,PeerChannel(channel_id=1109279145),NaN,Активисты «Бессмертного полка» возложили цветы...,photo,NaN,NaN,NaN,NaN,akitilop
2,15375,2022-12-03 18:56:42+00:00,512.0,NaN,PeerChannel(channel_id=1109279145),NaN,Как всегда наши учёные расширяют границы возмо...,video,5.0,NaN,NaN,NaN,akitilop
3,15374,2022-12-02 19:27:06+00:00,644.0,NaN,PeerChannel(channel_id=1109279145),NaN,Активисты общественных проектов и координаторы...,video,60.0,NaN,NaN,NaN,akitilop
4,15373,2022-12-02 18:58:20+00:00,705.0,NaN,PeerChannel(channel_id=1109279145),NaN,В Лужниках состоялся Форум боевых искусств «Би...,video,5.0,NaN,NaN,NaN,akitilop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14547,5,2017-02-01 10:56:33+00:00,356.0,NaN,PeerChannel(channel_id=1109279145),NaN,NaN,photo,NaN,NaN,NaN,NaN,akitilop
14548,4,2017-02-01 10:56:28+00:00,289.0,NaN,PeerChannel(channel_id=1109279145),NaN,NaN,photo,NaN,NaN,NaN,NaN,akitilop
14549,3,2017-02-01 10:51:25+00:00,278.0,NaN,PeerChannel(channel_id=1109279145),NaN,Приветствую вас мои маленькие любители политик...,text,NaN,NaN,NaN,NaN,akitilop
14550,2,2017-02-01 10:48:55+00:00,NaN,NaN,PeerChannel(channel_id=1109279145),NaN,NaN,photo,NaN,NaN,NaN,NaN,akitilop


In [150]:
test_df['msg_entity'][0]

'[<telethon.tl.types.MessageEntityItalic object at 0x153bef8e0>]'

In [151]:
test_df['fwd_from'][0]

'MessageFwdHeader(date=datetime.datetime(2022, 12, 1, 6, 2, 15, tzinfo=datetime.timezone.utc), imported=False, from_id=PeerChannel(channel_id=1029095426), from_name=None, channel_post=21757, post_author=None, saved_from_peer=None, saved_from_msg_id=None, psa_type=None)'

In [153]:
test_df2 = read_channel(test_df['frw_from_name'][0]+'.csv')
test_df2.head()

FileNotFoundError: [Errno 2] No such file or directory: './channels/DtRoad.csv'

In [155]:
test_df['fwd_from'][6]

'MessageFwdHeader(date=datetime.datetime(2022, 12, 2, 14, 37, 58, tzinfo=datetime.timezone.utc), imported=False, from_id=PeerChannel(channel_id=1394050290), from_name=None, channel_post=142568, post_author=None, saved_from_peer=None, saved_from_msg_id=None, psa_type=None)'

In [154]:
test_df2 = read_channel(test_df['frw_from_name'][6]+'.csv')
test_df2.head()

,id,date,views,reactions,to_id,fwd_from,message,type,duration,handle
0,144036,2022-12-20 09:25:01+00:00,20367.0,NaN,PeerChannel(channel_id=1394050290),NaN,❗️Смертность сельского населения из-за алкогол...,text,NaN,bbbreaking
1,144035,2022-12-20 09:20:12+00:00,49853.0,NaN,PeerChannel(channel_id=1394050290),NaN,❗️Число детей-инвалидов в России с 2019 года в...,text,NaN,bbbreaking
2,144034,2022-12-20 09:15:01+00:00,69298.0,NaN,PeerChannel(channel_id=1394050290),NaN,❗️Госдума во II чтении приняла законопроект о ...,text,NaN,bbbreaking
3,144033,2022-12-20 09:10:16+00:00,87720.0,NaN,PeerChannel(channel_id=1394050290),NaN,Украинский министр цифровой трансформации Миха...,text,NaN,bbbreaking
4,144032,2022-12-20 09:00:38+00:00,113195.0,NaN,PeerChannel(channel_id=1394050290),NaN,"Госдума приняла закон, устанавливающий огранич...",text,NaN,bbbreaking


In [156]:
test_df2['to_id'][0]

'PeerChannel(channel_id=1394050290)'

In [234]:
# from dateutil import parser
def process_fwd_from(row):
    if pd.isna(row['fwd_from']):
        return row

    # Define patterns to extract datetime, channel_id, and channel_post
    datetime_pattern = r'date=datetime\.datetime\((.*?)\)'
    channel_id_pattern = r'channel_id=(\d+)'
    channel_post_pattern = r'channel_post=(\d+)'

    # Use re.search to find matches in the input string
    datetime_match = re.search(datetime_pattern, row['fwd_from'])
    channel_id_match = re.search(channel_id_pattern, row['fwd_from'])
    channel_post_match = re.search(channel_post_pattern, row['fwd_from'])

    row['fwd_from_id'] = pd.NA
    row['fwd_mes_id'] = pd.NA


    # Extract values if matches are found
    if datetime_match:
        datetime_str = datetime_match.group(1)
        try:
            dt = datetime.strptime(datetime_str, '%Y, %m, %d, %H, %M, %S, tzinfo=datetime.timezone.utc')
            # dt = parser.parse(datetime_str)
            # row['datetime'] = dt
        except ValueError as e:
            try:
                dt = datetime.strptime(datetime_str, '%Y, %m, %d, %H, %M, tzinfo=datetime.timezone.utc')
                # print(datetime_str, dt)
            except ValueError as e:
                dt = np.NaN
                print(row)
                print(row['fwd_from'])
        row['fwd_date'] = dt

    if channel_id_match:
        # channel_id = channel_id_match.group(1)
        row['fwd_from_id'] = np.dtype('object').type(channel_id_match.group(1))

    if channel_post_match:
        # channel_post = channel_post_match.group(1)
        row['fwd_mes_id'] = np.dtype('object').type(channel_post_match.group(1))

    return row

In [227]:
# test_df_proc[['fwd_from_id', 'fwd_mes_id']] = pd.NA
# test_df_proc

,date,duration,frw_from_name,frw_from_title,fwd_date,fwd_from,fwd_from_id,fwd_mes_id,handle,id,message,msg_entity,reactions,to_id,type,views
0,2022-12-04 10:26:53+00:00,NaN,DtRoad,Дептранс Москвы,2022-12-01 06:02:15,"MessageFwdHeader(date=datetime.datetime(2022, ...",NaN,NaN,akitilop,15377,🚇БКЛ полноценно заработает уже в первом кварта...,[<telethon.tl.types.MessageEntityItalic object...,NaN,PeerChannel(channel_id=1109279145),photo,173.0
1,2022-12-03 19:32:06+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15376,Активисты «Бессмертного полка» возложили цветы...,NaN,NaN,PeerChannel(channel_id=1109279145),photo,528.0
2,2022-12-03 18:56:42+00:00,5.0,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15375,Как всегда наши учёные расширяют границы возмо...,NaN,NaN,PeerChannel(channel_id=1109279145),video,512.0
3,2022-12-02 19:27:06+00:00,60.0,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15374,Активисты общественных проектов и координаторы...,NaN,NaN,PeerChannel(channel_id=1109279145),video,644.0
4,2022-12-02 18:58:20+00:00,5.0,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15373,В Лужниках состоялся Форум боевых искусств «Би...,NaN,NaN,PeerChannel(channel_id=1109279145),video,705.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14547,2017-02-01 10:56:33+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,5,NaN,NaN,NaN,PeerChannel(channel_id=1109279145),photo,356.0
14548,2017-02-01 10:56:28+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,4,NaN,NaN,NaN,PeerChannel(channel_id=1109279145),photo,289.0
14549,2017-02-01 10:51:25+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,3,Приветствую вас мои маленькие любители политик...,NaN,NaN,PeerChannel(channel_id=1109279145),text,278.0
14550,2017-02-01 10:48:55+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,2,NaN,NaN,NaN,PeerChannel(channel_id=1109279145),photo,NaN


In [235]:
test_df_proc = test_df.apply(process_fwd_from, axis=1)

In [236]:
test_df_proc.head()

,date,duration,frw_from_name,frw_from_title,fwd_date,fwd_from,fwd_from_id,fwd_mes_id,handle,id,message,msg_entity,reactions,to_id,type,views
0,2022-12-04 10:26:53+00:00,NaN,DtRoad,Дептранс Москвы,2022-12-01 06:02:15,"MessageFwdHeader(date=datetime.datetime(2022, ...",1029095426,21757,akitilop,15377,🚇БКЛ полноценно заработает уже в первом кварта...,[<telethon.tl.types.MessageEntityItalic object...,NaN,PeerChannel(channel_id=1109279145),photo,173.0
1,2022-12-03 19:32:06+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15376,Активисты «Бессмертного полка» возложили цветы...,NaN,NaN,PeerChannel(channel_id=1109279145),photo,528.0
2,2022-12-03 18:56:42+00:00,5.0,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15375,Как всегда наши учёные расширяют границы возмо...,NaN,NaN,PeerChannel(channel_id=1109279145),video,512.0
3,2022-12-02 19:27:06+00:00,60.0,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15374,Активисты общественных проектов и координаторы...,NaN,NaN,PeerChannel(channel_id=1109279145),video,644.0
4,2022-12-02 18:58:20+00:00,5.0,NaN,NaN,NaT,NaN,NaN,NaN,akitilop,15373,В Лужниках состоялся Форум боевых искусств «Би...,NaN,NaN,PeerChannel(channel_id=1109279145),video,705.0


In [237]:
print(type(test_df_proc['fwd_date'][0]))
print(type(test_df_proc['fwd_from_id'][0]))
print(type(test_df_proc['fwd_mes_id'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
<class 'str'>


In [239]:
test_df_proc['fwd_from_id'] = test_df_proc['fwd_from_id'].astype('Int64')
test_df_proc['fwd_mes_id'] = test_df_proc['fwd_mes_id'].astype('Int64')
test_df_proc.head()

,date,duration,frw_from_name,frw_from_title,fwd_date,fwd_from,fwd_from_id,fwd_mes_id,handle,id,message,msg_entity,reactions,to_id,type,views
0,2022-12-04 10:26:53+00:00,NaN,DtRoad,Дептранс Москвы,2022-12-01 06:02:15,"MessageFwdHeader(date=datetime.datetime(2022, ...",1029095426,21757,akitilop,15377,🚇БКЛ полноценно заработает уже в первом кварта...,[<telethon.tl.types.MessageEntityItalic object...,NaN,PeerChannel(channel_id=1109279145),photo,173.0
1,2022-12-03 19:32:06+00:00,NaN,NaN,NaN,NaT,NaN,<NA>,<NA>,akitilop,15376,Активисты «Бессмертного полка» возложили цветы...,NaN,NaN,PeerChannel(channel_id=1109279145),photo,528.0
2,2022-12-03 18:56:42+00:00,5.0,NaN,NaN,NaT,NaN,<NA>,<NA>,akitilop,15375,Как всегда наши учёные расширяют границы возмо...,NaN,NaN,PeerChannel(channel_id=1109279145),video,512.0
3,2022-12-02 19:27:06+00:00,60.0,NaN,NaN,NaT,NaN,<NA>,<NA>,akitilop,15374,Активисты общественных проектов и координаторы...,NaN,NaN,PeerChannel(channel_id=1109279145),video,644.0
4,2022-12-02 18:58:20+00:00,5.0,NaN,NaN,NaT,NaN,<NA>,<NA>,akitilop,15373,В Лужниках состоялся Форум боевых искусств «Би...,NaN,NaN,PeerChannel(channel_id=1109279145),video,705.0


In [240]:
test_df_proc['fwd_from'][0]

'MessageFwdHeader(date=datetime.datetime(2022, 12, 1, 6, 2, 15, tzinfo=datetime.timezone.utc), imported=False, from_id=PeerChannel(channel_id=1029095426), from_name=None, channel_post=21757, post_author=None, saved_from_peer=None, saved_from_msg_id=None, psa_type=None)'

In [247]:
for i, channel in enumerate(channels):
    df = read_channel(channel)
    print(f"{i}/{len(channels)} {df['handle'][0]}")
    tqdm.pandas(desc = "Parsing reactions", unit='rows')
    df = process_reactions_df(df)
    tqdm.pandas(desc = "Parsing to_id", unit='rows')
    df = extract_to_id_df(df)
    tqdm.pandas(desc = "Parsing fwd_from", unit='rows')
    df = df.progress_apply(process_fwd_from, axis=1)
    # drop the fwd_from column
    df.drop(columns=['fwd_from'], inplace=True)
    try:
        df.drop(columns=['msg_entity'], inplace=True)
    except KeyError:
        pass
    save_channel(df, channel)

0/299 Abbasdjuma


Parsing fwd_from: 100%|██████████| 12002/12002 [00:03<00:00, 3014.79rows/s]


1/299 achexd2


Parsing fwd_from: 100%|██████████| 1454/1454 [00:00<00:00, 3426.93rows/s]


2/299 akashevarova


...

Parsing fwd_from: 100%|██████████| 126/126 [00:00<00:00, 2015.25rows/s]


297/299 znachit_net


Parsing fwd_from: 100%|██████████| 1710/1710 [00:00<00:00, 3648.85rows/s]


298/299 zvezdanews


Parsing fwd_from: 100%|██████████| 102190/102190 [00:12<00:00, 8141.99rows/s]


In [248]:
df.head()

,date,duration,fwd_date,fwd_from_id,fwd_mes_id,handle,id,message,to_id,type,views
0,2022-12-19 11:04:43+00:00,NaN,NaT,NaN,NaN,zvezdanews,103862,NaN,1144616980,photo,1577.0
1,2022-12-19 11:04:43+00:00,NaN,NaT,NaN,NaN,zvezdanews,103861,NaN,1144616980,photo,1577.0
2,2022-12-19 11:04:43+00:00,NaN,NaT,NaN,NaN,zvezdanews,103860,NaN,1144616980,photo,1577.0
3,2022-12-19 11:04:43+00:00,NaN,NaT,NaN,NaN,zvezdanews,103859,⚡️⚡️⚡️⚡️ Путин подписал закон о праве мобилизо...,1144616980,photo,1577.0
4,2022-12-19 10:57:43+00:00,NaN,NaT,NaN,NaN,zvezdanews,103858,❗️❗️❗️❗️Путин подписал закон о запрете на услу...,1144616980,text,2516.0
